
# Background
PSI/J (Portable Submission Interface for Jobs), is a Python abstraction layer which allows your HPC application to run virtually anywhere. <br> <br>
![psij-overview-visual](./images/psij-overview.png)
<br>
<br>
PSI/J automatically translates abstract job specifications into concrete scripts and commands to send to the scheduler. PSI/J has a number of advantages:
1. **Runs entirely in user space**: no need to wait for infrequent deployment cycles, it's easy to leverage built-in or community-provided plugins
2. **An asynchronous modern API for job management**: a clean Python API for requesting and managing jobs.
3. **Supports the common batch schedulers**: we test PSI/J across multiple DOE supercomputer centers. It’s easy to test PSI/J on your systems and share the results with the community.
4. **Built by the HPC community, for the HPC community**: PSI/J is based on a number of libraries used by state-of-the-art HPC workflow applications.
5. **PSI/J is an open source project**: we are establishing a community to develop, test, and deploy PSI/J across many HPC facilities.

## Install from pip ...

In [ ]:
!bash -c "pip install psij-python"

## ... Or Install from Source

In [ ]:
!bash -c "python3 --version"
!bash -c "python3 -m venv psij-tutorial-venv"
!bash -c "source ./psij-tutorial-venv/bin/activate"
!bash -c "pip install --upgrade pip"
!bash -c "pip install git+https://github.com/ExaWorks/psij-python.git"

# Overview 
In PSI/J’s terminology, a [Job](https://exaworks.org/psij-python/docs/.generated/psij.html#psij.job.Job) represents an executable plus a bunch of attributes. Static job attributes such as resource requirements are defined by the [JobSpec]() at creation. Dynamic job attributes such as the [JobState]() are modified by [JobExecutors]() as the [Job]() progresses through its lifecycle.

A [JobExecutor]() represents a specific *Resource Manager*, e.g. Slurm, on which the Job is being executed. Available *Resource Managers*: 
- Local
- Slurm
- LSF
- Cobalt
- PBS

Generally, when jobs are submitted, they will be queued for a variable period of time, depending on how busy the target machine is. Once the Job is started, its executable is launched and runs to completion.

In PSI/J, a job is submitted by [JobExecutor.submit(Job)]() which permanently binds the Job to that executor and submits it to the underlying resource manager.

# Basic Usage
Creating a *single* job with PSI/J:
1. Create a [JobExecutor]() instance.
2. Create a [JobSpec]() object and populate it with information about your job
3. Create a [Job]() with that [JobSpec]()
4. Submit the [Job]() instance to the [Job](Executor)

## Viewing Job Status
To view a jobs' status, you can use `print(str(Job.status))`

In [2]:
import time
from psij import Job, JobExecutor, JobSpec

environment = "local"
executable = "/bin/date"
ex = JobExecutor.get_instance(environment)
job = Job(JobSpec(executable=executable))

print("Job Status Stages:")
print("\tPrior to submitting: ", str(job.status))
ex.submit(job)
print("\tAfter submitting: ", str(job.status))
time.sleep(1)
print("\tOnce completed: ", str(job.status))

Job Status Stages:
	Prior to submitting:  JobStatus[NEW, time=1673041703.5975358]
	After submitting:  JobStatus[ACTIVE, time=1673041703.6158385]
	Once completed:  JobStatus[COMPLETED, time=1673041703.7166197, exit_code=0]


### Multiple Jobs
This section will show you how to submit *multiple* jobs

In [3]:
from psij import Job, JobExecutor, JobSpec
from psij import JobState

executable = "/bin/date"
ex = JobExecutor.get_instance(environment)

jobs = []

print("Submitting jobs: ")
for _ in range(10):
    job = Job(JobSpec(executable=executable))
    print(str(job.status))
    jobs.append(job)
    ex.submit(job)

time.sleep(1)
print("\nCompleted Jobs: ")
_ = [print(str(job.status)) for job in jobs]

Submitting jobs: 
JobStatus[NEW, time=1673041705.85077]
JobStatus[NEW, time=1673041705.868976]
JobStatus[NEW, time=1673041705.8858225]
JobStatus[NEW, time=1673041705.9006584]
JobStatus[NEW, time=1673041705.915577]
JobStatus[NEW, time=1673041705.9316385]
JobStatus[NEW, time=1673041705.9460511]
JobStatus[NEW, time=1673041705.9592762]
JobStatus[NEW, time=1673041705.9721496]
JobStatus[NEW, time=1673041705.984657]

Completed Jobs: 
JobStatus[COMPLETED, time=1673041705.9202194, exit_code=0]
JobStatus[COMPLETED, time=1673041705.9203172, exit_code=0]
JobStatus[COMPLETED, time=1673041705.9246562, exit_code=0]
JobStatus[COMPLETED, time=1673041706.1285424, exit_code=0]
JobStatus[COMPLETED, time=1673041706.1286585, exit_code=0]
JobStatus[COMPLETED, time=1673041706.1287165, exit_code=0]
JobStatus[COMPLETED, time=1673041706.1287718, exit_code=0]
JobStatus[COMPLETED, time=1673041706.1288874, exit_code=0]
JobStatus[COMPLETED, time=1673041706.128965, exit_code=0]
JobStatus[COMPLETED, time=1673041706.12

Every instance of `JobExecutor` can handle an abitrar number of submitted jobs. For reference, we have tested it with up to 64,000 jobs.

## Waiting for Completion
To wait for a job to complete once it has been submitted, it suffices to call the [wait]() method with no arguments:

In [4]:
from psij import Job, JobSpec

job = Job(JobSpec(executable="/bin/date"))
ex.submit(job)
job.wait()

The [wait]() call will return once the job has reached a terminal state, which almost always means that it finished or was cancelled.

To distinguish jobs that complete successfully from ones that fail or are cancelled, fetch the status of the job after calling [wait]():

In [5]:
job.wait()
print(str(job.status))

JobStatus[COMPLETED, time=1673041709.5348377, exit_code=0]


# Configuring a Job
The previous examples specify `executable='/bin/date'` tells PSI/J that we want to run the `/bin/date` command. The superclass is `JobSpec`, which allows you to configure the following parameters used in the instance of `Job`:
- arguements for teh job executable
- environment the job is runnning in
- destination for standard output and error streams
- resource requirements for the job's execution
- accounting details to be used

Relevant Docs: [JobSpec]()

## Job Arguements
The executable’s command line arguments to be used for a job are specified as a list of strings in the arguments attribute of the `JobSpec` class. For example, our previous /bin/date job could be changed to request UTC time formatting:

In [6]:
environment = 'local'
ex = JobExecutor.get_instance(environment)
job = Job(JobSpec(executable='/bin/date', arguments=['-utc', '--debug']))

print(str(job.status))
ex.submit(job)
job.wait()
print(str(job.status))

JobStatus[NEW, time=1673041714.1849961]
JobStatus[FAILED, time=1673041714.3477576, message=_PSI_J_LAUNCHER_DONE, exit_code=1]


Note: *JobSpec* can also be added incrementally

In [7]:
from psij import JobSpec

spec = JobSpec()
spec.executable = '/bin/date'
spec.arguments = ['-u']
job = Job(spec)

print(str(job.status))
ex.submit(job)
job.wait()
print(str(job.status))

JobStatus[NEW, time=1673041716.008083]
JobStatus[COMPLETED, time=1673041716.1515958, exit_code=0]


## Job Environment
The job environment sets the environment variables for a job before it is launched. This is the equivalent of exporting `FOO=bar` on the command line before running a command. These environment variables are specified as a dictionary of string key/value pairs:

*Note: Environment variables specified this way will overwrite settings from your shell initialization files (e.g., ~/.bashrc), including from any modules loaded in the default shell environment.*

In [8]:
from psij import JobSpec

spec = JobSpec()
spec.executable = '/bin/date'
spec.environment = {'TZ': 'America/Los_Angeles'}
job = Job(spec)

print(str(job.status))
ex.submit(job)
job.wait()
print(str(job.status))

JobStatus[NEW, time=1673041717.778529]
JobStatus[COMPLETED, time=1673041717.9551513, exit_code=0]


## Job Stdio
Standard output and standard error streams of the job can be individually redirected to files by setting the stdout_path and stderr_path attributes:

The job’s standard input stream can also be redirected to read from a file, by setting the `spec.stdin_path` attribute.

In [9]:
from psij import JobSpec

spec = JobSpec()
spec.executable = '/bin/date'
spec.stdout_path = '/tmp/date.out'
spec.stderr_path = '/tmp/date.err'
job = Job(spec)

print(str(job.status))
ex.submit(job)
job.wait()
print(str(job.status))

JobStatus[NEW, time=1673041719.5712633]
JobStatus[COMPLETED, time=1673041719.7589772, exit_code=0]


## Job Resources
A job submitted to a cluster is allocated a specific set of resources to run on. The amount and type of resources are defined by a resource specification `ResourceSpec` which becomes a part of the job specification. The resource specification supports the following attributes:

* `node_count`: allocate that number of compute nodes to the job. All cpu-cores and gpu-cores on the allocated node can be exclusively used by the submitted job.

* `processes_per_node`: on the allocated nodes, execute that given number of processes.

* `process_count`: the total number of processes (MPI ranks) to be started

* `cpu_cores_per_process`: the number of cpu cores allocated to each launched process. PSI/J uses the system definition of a cpu core which may refer to a physical cpu core or to a virtual cpu core, also known as a hardware thread.

* `gpu_cores_per_process`: the number of gpu cores allocated to each launched process. The system definition of an gpu core is used, but usually refers to a full physical GPU.

* `exclusive_node_use`: When this boolean flag is set to True, then PSI/J will ensure that no other jobs, neither of the same user nor of other users of the same system, will run on any of the compute nodes on which processes for this job are launched.

A resource specification does not need to define all available attributes. In fact, an empty resource spec is valid as it refers to a single process being launched on a single cpu core.

The user should also take care not to define contradictory statements. For example, the following specification cannot be enacted by PSI/J as the specified node count contradicts the value of `process_count / processes_per_node`:

In [10]:
from psij import JobSpec, ResourceSpecV1

spec = JobSpec()
spec.executable = '/bin/stress'
spec.resource_spec = ResourceSpecV1(node_count=2, processes_per_node=2,
        process_count=4)
job = Job(spec)

print(str(job.status))
ex.submit(job)
job.wait()
print(str(job.status))

JobStatus[NEW, time=1673041721.837711]
JobStatus[FAILED, time=1673041721.9633806, message=_PSI_J_LAUNCHER_DONE, exit_code=127]


# Processes versus ranks
All processes of the job will share a single MPI communicator (*MPI_COMM_WORLD*), independent of their placement, and the term *rank* (which usually refers to an MPI rank) is thus equivalent. However, jobs started with a single process instance may, depending on the executor implementation, not get an MPI communicator. How Jobs are launched can be specified by the launcher attribute of the `JobSpec`, as documented below.

## Launching Methods
To specify how the processes in your job should be started once resources have been allocated for it, pass the name of a launcher (e.g. `"mpirun"`, `"srun"`, etc.) like so: `JobSpec(..., launcher='srun')`.

## Scheduling Information
To specify resource-manager-specific information, like queues/partitions, runtime, and so on, create a [JobAttributes]() and set it with `JobSpec(..., attributes=my_job_attributes)`:

In [11]:
from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1

environment = 'local'
ex = JobExecutor.get_instance(environment)

job = Job(
    JobSpec(
        executable="/bin/date",
        resources=ResourceSpecV1(node_count=1),
        attributes=JobAttributes(
            queue_name="<QUEUE_NAME>", project_name="<ALLOCATION>"
        ),
    )
)

print(str(job.status))
ex.submit(job)
job.wait()
print(str(job.status))


JobStatus[NEW, time=1673041724.589741]
JobStatus[COMPLETED, time=1673041724.7682035, exit_code=0]


Note: The <QUEUE_NAME> and <ALLOCATION> fields will depend on the system you are running on.

# Managing Job State
In all the above examples, we have submitted jobs without checking on what happened to them. Once the job has finished executing (which, for */bin/date*, should be almost as soon as the job starts) the resource manager will mark the job as complete, triggering PSI/J to do the same via the [JobStatus]() attribute of the job. `Job` state progressions follow this state model:

<img src="./images/job_states.png" width="550"/>


## Cancelling a Job
If supported by the underlying job scheduler, PSI/J jobs can be canceled by invoking the [cancel]() method.

In [12]:
job = Job(JobSpec(executable="/bin/date"))
ex.submit(job)
job.cancel()
print(str(job.status))

JobStatus[CANCELED, time=1673041727.8790166]


## Status Callbacks
Waiting for jobs to complete with [wait]() is fine if you don’t mind blocking while you wait for a single job to complete. However, if you want to wait on multiple jobs without blocking, or you want to get updates when jobs start running, you can attach a callback to a [JobExecutor]() which will fire whenever any job submitted to that executor changes status.

To wait on multiple jobs at once:

import time
from psij import Job, JobExecutor, JobSpec

count = 3

def callback(job, status):
    global count

    if status.final:
        print(f"Job {job} completed with status {status}\n")
        count -= 1

ex = JobExecutor.get_instance("local")
ex.set_job_status_callback(callback)

for _ in range(count):
    job = Job(JobSpec(executable="/bin/date"))
    ex.submit(job)

while count > 0:
    time.sleep(0.01)

Status callbacks can also be set on individual jobs with [set_job_status_callback()]().